In [1]:
import re
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import bs4
import ssl
import urllib3
import pandas as pd
import faiss
import os
from langchain.embeddings import HuggingFaceEmbeddings
import torch
import gc

def instruct_structure(prompt):
    input_text, output_text = prompt.split('### target')
    input_text = input_text.replace('### glossaries', '### glossary').replace('\n* ', '\n• ')
    input_text = re.sub(r"\[[^\]]+\] ", "[UNK] ", input_text)
    return input_text


project_id = "prod-ai-project"

from google.cloud import bigquery
client = bigquery.Client(project=project_id)
sql = """select series_id, episode_id, org_input_text, org_output_text, prompt 
        from webtoon_translation.structured_240820_ep_line
        where data_split = 'romance_valid'"""
df = client.query(sql).result().to_dataframe()
from tqdm import tqdm
tqdm.pandas()
df['prompt'] = df['prompt'].progress_apply(lambda x: instruct_structure(x))

/home/bun.2/.cache/pypoetry/virtualenvs/poetry-env-eDEwtiIl-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 18885.21it/s]


In [32]:
data_idx = 58
data = df['prompt'][data_idx]
example = data.split("### source")[1].strip()
print(data)

### glossary
• 시그렌 (M): siegren
• 유니스 (F): eunice
• 피오나 (F): fiona
• 마법사: mage / sorcerer
• 하녀: maid

### source
000	[UNK] 『금일 오후,
001	[UNK] 온실에서 있을 티 파티에 영애를 초대하고 싶습니다.』
002	[UNK] 『시간이 나신다면 꼭 참석해 주세요,
003	[UNK] 헤더 아덴 자작 영애.』
004	[UNK] 이 사람,
005	[UNK] 어제 시그렌이랑 일이 있었던 그 영애 아냐?
006	[UNK] 그런 사람이 여는 티파티에 내가 가도 되는 건가?
007	[UNK] 내가 괜히 신경 쓸 만한 이유는 없지.
008	[UNK] 잘못한 건 그쪽이고.
009	[UNK] 어쩌면 티파티에서 유니스를 만날 수 있을지도 모르니까.
010	[UNK] 기쁜 마음으로 응하겠다고 전해줘.
011	[UNK] 유니스 영애.
012	[UNK] 피오나 영애…!
013	[UNK] 유니스 영애도 초대받은 건가요?
014	[UNK] 네.
015	[UNK] 피오나 영애와는 같은 일행임에도 계속 얼굴을 볼 수가 없었네요.
016	[UNK] 전 말을 타고 있었거든요.
017	[UNK] …그래도 마차에도 와 주세요.
018	[UNK] 혼자서는 심심해서….
019	[UNK] 주변에 온통 기사랑 병사라서 쓸쓸했나 봐.
020	[UNK] 어서 오십시오.
021	[UNK] 이쪽이 온실입니다.
022	[UNK] 아가씨께서는 안쪽에서 기다리고 계십니다.
023	[UNK] 어머나,
024	[UNK] 피오나 영애,
025	[UNK] 유니스 영애…
026	[UNK] 초대에 응해주셔서 감사해요.
027	[UNK] 기다리고 있었답니다.
028	[UNK] 와, 역시 화려한 미인이야.
029	[UNK] 이런 사람이 시그렌에게 달려들었단 말이지.
030	[UNK] 자, 어서 앉으세요.
031	[UNK] 네, 고마워요.
032	[UNK] 이쪽은 근처 영지의 영애들이에요.
033	[UNK] 명성이 자자한 두 분을 만나서 

In [29]:
SYSTEM_PROMPT = {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": """You're an expert translator who translates Korean webtoon in English. Make sure the number of target sentences matches the number of source sentences. The result should be TSV formatted. 
            • Find a balance between staying true to the Korean meaning and keeping a natural flow. Don't be afraid to add to the text. Embellish it. 
            • Avoid translating word-for-word. Keep the general feeling and translate the text accordingly. 
            • Translate with an American audience in mind. This means easy-to-read, conversational English.""",
                }
            ],
        }

chat_completion = openai_client.beta.chat.completions.parse(
    model= GPT_FINE_TUNING_MODEL,
    messages = [
        SYSTEM_PROMPT,
        {
            "role":"user",
            "content" : [{"type" : "text",
                          "text" : data
                        }],
        }
    ],
    temperature= 0.2,
    top_p = 0.8
)
response = chat_completion.choices[0].message.content

In [30]:
print(response)

000	“I would like to invite you, lady fiona, to a tea party…
001	…that will be held in the greenhouse this afternoon.”
002	“Please attend if you have time,
003	lady fiona.”
004	wait, isn’t this…
005	…from the lady who caused that scene with siegren yesterday?
006	should I really be going to a tea party hosted by someone like that?
007	well, there’s no reason for me to worry about it.
008	she’s the one who did something wrong.
009	and I might be able to see eunice there.
010	please tell her that I’ll be happy to attend.
011	lady eunice.
012	lady fiona…!
013	did you receive an invitation too?
014	yes.
015	we’re traveling together, but I haven’t been able to see you.
016	I’ve been riding a horse.
017	please come to the carriage too…
018	it’s boring being alone…
019	she must have felt lonely being surrounded by only knights and soldiers.
020	welcome.
021	this way to the greenhouse.
022	my lady is waiting inside.
023	oh my,
024	lady fiona,
025	lady eunice…
026	thank you for accepting my inv

In [33]:
response_split = response.split('\n')
source = data.split("### source")[1].strip().split('\n')

In [37]:
print(source)

['000\t[UNK] 『금일 오후,', '001\t[UNK] 온실에서 있을 티 파티에 영애를 초대하고 싶습니다.』', '002\t[UNK] 『시간이 나신다면 꼭 참석해 주세요,', '003\t[UNK] 헤더 아덴 자작 영애.』', '004\t[UNK] 이 사람,', '005\t[UNK] 어제 시그렌이랑 일이 있었던 그 영애 아냐?', '006\t[UNK] 그런 사람이 여는 티파티에 내가 가도 되는 건가?', '007\t[UNK] 내가 괜히 신경 쓸 만한 이유는 없지.', '008\t[UNK] 잘못한 건 그쪽이고.', '009\t[UNK] 어쩌면 티파티에서 유니스를 만날 수 있을지도 모르니까.', '010\t[UNK] 기쁜 마음으로 응하겠다고 전해줘.', '011\t[UNK] 유니스 영애.', '012\t[UNK] 피오나 영애…!', '013\t[UNK] 유니스 영애도 초대받은 건가요?', '014\t[UNK] 네.', '015\t[UNK] 피오나 영애와는 같은 일행임에도 계속 얼굴을 볼 수가 없었네요.', '016\t[UNK] 전 말을 타고 있었거든요.', '017\t[UNK] …그래도 마차에도 와 주세요.', '018\t[UNK] 혼자서는 심심해서….', '019\t[UNK] 주변에 온통 기사랑 병사라서 쓸쓸했나 봐.', '020\t[UNK] 어서 오십시오.', '021\t[UNK] 이쪽이 온실입니다.', '022\t[UNK] 아가씨께서는 안쪽에서 기다리고 계십니다.', '023\t[UNK] 어머나,', '024\t[UNK] 피오나 영애,', '025\t[UNK] 유니스 영애…', '026\t[UNK] 초대에 응해주셔서 감사해요.', '027\t[UNK] 기다리고 있었답니다.', '028\t[UNK] 와, 역시 화려한 미인이야.', '029\t[UNK] 이런 사람이 시그렌에게 달려들었단 말이지.', '030\t[UNK] 자, 어서 앉으세요.', '031\t[UNK] 네, 고마워요.', '032\t[UNK] 이쪽은 근처 영지의 영애들이에요.', '033\t[

In [38]:
print(response_split[0])

000	“I would like to invite you, lady fiona, to a tea party…


In [39]:
GPT_BASE_MODEL = "chatgpt-4o-latest"

SYSTEM_REVIEW_PROMPT = {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "너는 1차 번역된 웹툰 대화를 검토하고 이를 수정하거나 아예 재생성하는 태스크를 수행할 거야. \
                    먼저 맥락을 이해하기 위해 한 회차의 [한국어 대화문]이 주어지고, \
                    그 중에서 맨 마지막 두 줄에 [한글 대화]와 이 대화에 대응되는 [1차 영어 번역문]이 주어질 거야. \
                    먼저 [한국어 대화문]속의 [한글 대화]가 앞 대사 혹은 뒷 대사와 이어지는 문장인지 검토해. \
                    특히 웹툰은 '...'을 대사 앞에 붙이면 앞 문장과 이어지는 것을 나타내고, '...'을 대사 뒤에 붙이면 뒷 문장과 이어지는 것을 나타내기 때문에 이를 힌트로 활용하면 좋아. \
                    그리고 ','도 뒷 대사와 이어지는 힌트가 될 수 있기 때문에 해당 부분도 유념해서 보면 좋아. \
                    따라서 [한글 대화]가 만약 앞 대사와 혹은 뒷 대사와 이어지는 문장이라면 [1차 영어 번역문]을 \
                    '<step1> [1차 영어 번역문] </step1>'와 같이 출력하고 생성을 종료해.\n\
                    하지만 [한글 대화]가 앞 대사나 뒷 대사와 이어지는 대사가 아니라면 \
                    그 다음 단계로 [1차 영어 번역문]을 한글로 번역해서 [1차 한글 번역문]을 생성하고, [한글 대화]의 의미가 대체로 일치하는지 비교해.\n\
                    만약 의미가 일치한다면 원래의 [1차 영어 번역문]을 똑같이 생성해도 돼. \
                    하지만 만약 [1차 한글 번역문]과 [한글 대화]의 의미가 일치하지 않거나 정보의 양이 다르다면 \
                    [1차 영어 번역문]과 [1차 한글 번역문]은 무시하고 [한글 대화]에 대응하는 영어 번역문을 생성해.\
                    예를 들어 [한글 대화]가 '사과'인데 [1차 한글 번역문]이 '붉은 사과'라면 \
                    이는 [한글 대화]에 [1차 한글 번역문]의 일부만 제공된 상태이기 때문에 정보의 양이 다른 것으로 판단해. \
                    하지만 예외적으로 [한글 대화]가 '달렸다'인데 [1차 한글 번역문]이 '주어가 달렸다'라면 \
                    이는 정보의 양이 같은 것으로 판단해. \
                    왜냐하면 웹툰은 영어로 번역할 때 생략된 주어도 넣어줘야하기 때문에 주어의 유무는 정보의 양이 다른 걸로 판단하지 않아.\
                    그리고 웹툰은 '...'을 대사 앞에 붙이면 앞 문장과 이어지는 것을 나타내고, '...'을 대사 뒤에 붙이면 뒷 문장과 이어지는 것을 나타내기 때문에 \
                    해당 구성요소의 차이도 정보의 양이 다르다고 볼 수 있으니 이를 유념해줘.\n\
                    따라서 [1차 한글 번역문]과 [한글 대화]의 주어가 불일치 하거나, 주어가 아닌 다른 문장 구성 요소의 정보의 양이 다르다면 \
                    [1차 한글 번역문]과 [1차 영어 번역문]은 무시하고 [한글 대화]만을 가지고 다시 번역하면 돼. [한글 대화]의 '...'도 번역할 때 제거하지마.\
                    그리고 이 대사의 주어가 무엇인지도 꼭 생각하고, 만약 [한글 대화]에 서술어만 존재한다면 주어를 유추해서 추가한 이후에 번역해줘.\
                    그리고 새롭게 생성한 영어 번역문은 <step1>으로 시작하고 </step1>으로 끝내. \
                    만약 추론이 필요하다면 <reasoning>으로 시작해서 추론 내용을 입력하고 </reasoning>으로 마무리해.\
                    즉 여기까지의 출력 형태는 '<reasoning> ...추론 내용... </reasoning> <step1> 1단계 생성 번역문 </step1>' 혹은 \
                    <step1> 1단계 생성 번역문 </step1>가 될거야"
                }
            ],
}


In [40]:
import re

def clean_text(text):
    return re.sub(r'^\d+\s+\[UNK\]\s+', '', text)
def clean_text2(text):
    return re.sub(r'^\d+\s+','', text)

def extract_translation(text):
    match = re.search(r'<translate>(.*?)</translate>', text, re.DOTALL)
    return match.group(1).strip() if match else None

review_li = []
for i, r in enumerate(response_split):
    # if i == 10:#10줄만 스캔
    #     break
    review_text = f"{input_text}\n\n### review\n[한글 대화] {source[i]}\n[1차 영어 번역문] {clean_text2(r)}\n\n" 
    # if i == 0:
    #     print(review_text)
    review_completion = openai_client.beta.chat.completions.parse(
        model= GPT_BASE_MODEL,
        messages = [
            SYSTEM_REVIEW_PROMPT,
            {
                "role":"user",
                "content" : [{"type" : "text",
                              "text" : review_text
                            }],
            }
        ],
        temperature= 0.2,
        top_p = 0.8
    )
    review = review_completion.choices[0].message.content
    print(review)
    review_li.append(extract_translation(review))

<step1> “I would like to invite you, Lady Fiona, to a tea party this afternoon. </step1>
<step1> …I would like to invite you to the tea party that will be held in the greenhouse this afternoon. </step1>
<step1> “Please attend if you have time, </step1>
<reasoning>  
[한글 대화] "헤더 아덴 자작 영애."는 앞 문장인 "시간이 나신다면 꼭 참석해 주세요,"와 이어지는 문장입니다. 이는 초대장의 문맥에서 "시간이 나신다면 꼭 참석해 주세요, 헤더 아덴 자작 영애."라는 한 문장으로 구성되어 있으며, 문장 끝에 붙은 '』' 기호는 인용문의 끝을 나타냅니다. 따라서 이 대사는 앞 문장과 이어지는 문장입니다.  
</reasoning>  
<step1> lady fiona.” </step1>
<step1> wait, isn’t this… </step1>
<step1> …from the lady who caused that scene with Siegren yesterday? </step1>
<step1> should I really be going to a tea party hosted by someone like that? </step1>
<step1> well, there’s no reason for me to worry about it. </step1>


KeyboardInterrupt: 

In [92]:
#assert len(response_split) == len(review_li) == len(source)
def clean_text(text):
    return re.sub(r'^\d+\s+\[UNK\]\s+', '', text)
def clean_text2(text):
    return re.sub(r'^\d+\s+','', text)
for i in range(10):#len(response_split)):
    hangle = clean_text(source[i])
    print(f'[한글]{hangle}')
    first_translate = clean_text2(response_split[i])
    print(f'[기존]{first_translate}')
    second_translate = review_li[i]
    print(f'[리뷰]{second_translate}')
    print()

[한글]『금일 오후,
[기존]“my lady, I would like to invite you to a tea party in the greenhouse this afternoon.”
[리뷰]"This afternoon,"

[한글]온실에서 있을 티 파티에 영애를 초대하고 싶습니다.』
[기존]“my lady, I would like to invite you to a tea party in the greenhouse this afternoon.”
[리뷰]“My lady, I would like to invite you to the tea party that will be held in the greenhouse.”

[한글]『시간이 나신다면 꼭 참석해 주세요,
[기존]“please attend if you have time,
[리뷰]Please make sure to attend if you have time.

[한글]헤더 아덴 자작 영애.』
[기존]lady heather adens.”
[리뷰]Lady Heather Aden.

[한글]이 사람,
[기존]isn’t she...
[리뷰]That person,

[한글]어제 시그렌이랑 일이 있었던 그 영애 아냐?
[기존]...the lady who was with siegren yesterday?
[리뷰]Isn't she the lady who had an incident with Siegren yesterday?

[한글]그런 사람이 여는 티파티에 내가 가도 되는 건가?
[기존]should I really go to a tea party hosted by someone like that?
[리뷰]Am I even allowed to attend a tea party hosted by someone like that?

[한글]내가 괜히 신경 쓸 만한 이유는 없지.
[기존]well, I guess there’s no reason for me to be worried.
[리뷰]There’s no reason for 